# Notebook 0: Classes de Base pour la Résolution de Sudoku

Ce notebook contient les classes de base nécessaires pour la manipulation et la résolution des grilles de Sudoku. Il sera importé dans les autres notebooks pour réutiliser ces classes.

## Importation des Bibliothèques Nécessaires

Nous commençons par importer les bibliothèques nécessaires.


In [ ]:
using System;
using System.Collections.Generic;
using System.Collections.ObjectModel;
using System.IO;
using System.Linq;
using System.Text;


## Définition de la classe SudokuGrid

Nous définissons ici la classe SudokuGrid qui représente une grille de Sudoku et fournit des méthodes pour manipuler et afficher les grilles.


In [ ]:
public class SudokuGrid : ICloneable
{

    // Méthodes utilitaires
    public static T[][] ToJaggedArray<T>(IList<T> source, int columnLength)
    {
        return source
            .Select((value, index) => new { value, index })
            .GroupBy(x => x.index / columnLength)
            .Select(g => g.Select(x => x.value).ToArray())
            .ToArray();
    }

    public static T[] Flatten<T>(T[][] source) => source.SelectMany(x => x).ToArray();

    public static T[,] To2D<T>(T[][] source)
    {
        int rowLength = source.Length;
        int colLength = source[0].Length;
        var result = new T[rowLength, colLength];

        for (int row = 0; row < rowLength; row++)
            for (int col = 0; col < colLength; col++)
                result[row, col] = source[row][col];

        return result;
    }

    public static T[][] ToJaggedArray<T>(T[,] source)
    {
        int rows = source.GetLength(0);
        int cols = source.GetLength(1);
        var result = new T[rows][];

        for (int i = 0; i < rows; i++)
        {
            result[i] = new T[cols];
            for (int j = 0; j < cols; j++)
                result[i][j] = source[i, j];
        }

        return result;
    }

    public static readonly ReadOnlyCollection<int> NeighbourIndices = new(Enumerable.Range(0, 9).ToList());

    private static readonly (int row, int column)[][] LineNeighbours = NeighbourIndices.Select(r => NeighbourIndices.Select(c => (r, c)).ToArray()).ToArray();
    private static readonly (int row, int column)[][] ColumnNeighbours = NeighbourIndices.Select(c => NeighbourIndices.Select(r => (r, c)).ToArray()).ToArray();
    private static readonly (int row, int column)[][] BoxNeighbours = GetBoxNeighbours();
    public static readonly (int row, int column)[][] AllNeighbours = LineNeighbours.Concat(ColumnNeighbours).Concat(BoxNeighbours).ToArray();

    private static (int row, int column)[][] GetBoxNeighbours()
    {
        var result = new (int row, int column)[9][];

        for (int box = 0; box < 9; box++)
        {
            var cells = new List<(int, int)>();
            int startRow = (box / 3) * 3;
            int startCol = (box % 3) * 3;

            for (int row = startRow; row < startRow + 3; row++)
                for (int col = startCol; col < startCol + 3; col++)
                    cells.Add((row, col));

            result[box] = cells.ToArray();
        }

        return result;
    }

    public static readonly (int row, int column)[][][] CellNeighbours = NeighbourIndices
        .Select(row => NeighbourIndices
            .Select(col => AllNeighbours
                .Where(neighbourhood => neighbourhood.Contains((row, col)))
                .SelectMany(n => n)
                .Where(pos => pos != (row, col))
                .Distinct()
                .ToArray())
            .ToArray())
        .ToArray();

    public SudokuGrid() { }

    public int[,] Cells { get; set; } = new int[9, 9];

    public override string ToString()
    {
        var output = new StringBuilder();
        var lineSep = new string('-', 31);
        var blankSep = new string(' ', 8);

        for (int row = 0; row < 9; row++)
        {
            output.Append(row % 3 == 0 ? lineSep + "\n" : "");
            output.Append("| ");
            for (int col = 0; col < 9; col++)
            {
                output.Append(Cells[row, col] > 0 ? Cells[row, col].ToString() : " ");
                output.Append((col + 1) % 3 == 0 ? " | " : "  ");
            }
            output.Append("\n");
        }
        output.Append(lineSep);

        return output.ToString();
    }

    public int[] GetAvailableNumbers(int x, int y)
    {
        if (x < 0 || x >= 9 || y < 0 || y >= 9)
            throw new ApplicationException("Invalid Coordinates");

        bool[] used = new bool[9];
        foreach (var (row, col) in CellNeighbours[x][y])
        {
            int value = Cells[row, col];
            if (value > 0) used[value - 1] = true;
        }

        return Enumerable.Range(1, 9).Where(n => !used[n - 1]).ToArray();
    }

    public static SudokuGrid ReadSudoku(string sudokuAsString) => ReadMultiSudoku(new[] { sudokuAsString })[0];

    public static List<SudokuGrid> ReadSudokuFile(string fileName) => ReadMultiSudoku(File.ReadAllLines(fileName));


    public static List<SudokuGrid> ReadMultiSudoku(string[] lines)
    {
        var toReturn = new List<SudokuGrid>();
        var rows = new List<int[]>();
        var rowCells = new List<int>(9);

        foreach (var line in lines.Where(l => l.Length > 0 && IsSudokuChar(l[0])))
        {
            foreach (char c in line)
            {
                if (IsSudokuChar(c))
                {
                    if (char.IsDigit(c))
                    {
                        rowCells.Add((int)Char.GetNumericValue(c));
                    }
                    else
                    {
                        rowCells.Add(0);
                    }
                }

                if (rowCells.Count == 9)
                {
                    rows.Add(rowCells.ToArray());
                    rowCells.Clear();
                }

                if (rows.Count == 9)
                {
                    toReturn.Add(new SudokuGrid() { Cells = To2D(rows.ToArray()) });
                    rows.Clear();
                }
            }
        }

        return toReturn;
    }

     private static bool IsSudokuChar(char c) => char.IsDigit(c) || c == '.' || c == 'X' || c == '-';

    public object Clone() => new SudokuGrid { Cells = (int[,])Cells.Clone() };

    public int NbErrors(SudokuGrid originalPuzzle)
    {
        int errors = AllNeighbours.Select(n => n.Select(pos => Cells[pos.row, pos.column])
                                                 .GroupBy(val => val)
                                                 .Sum(g => g.Count() - 1))
                                   .Sum();

        foreach (var row in NeighbourIndices)
            foreach (var col in NeighbourIndices)
                if (originalPuzzle.Cells[row, col] > 0 && originalPuzzle.Cells[row, col] != Cells[row, col])
                    errors++;

        return errors;
    }

    public bool IsValid(SudokuGrid originalPuzzle) => NbErrors(originalPuzzle) == 0;

    public int NbEmptyCells() => Cells.Cast<int>().Count(c => c == 0);
}


## Définition de l'interface ISudokuSolver

Nous définissons ici l'interface ISudokuSolver qui sera implémentée par les différentes stratégies de résolution de Sudoku.


In [ ]:
public interface ISudokuSolver
{
    SudokuGrid Solve(SudokuGrid s);
}


## Définition de la classe SudokuHelper

Nous ajoutons ici la classe SudokuHelper qui contient des méthodes utilitaires pour charger et afficher des grilles de Sudoku.


In [ ]:
using System.IO;

public enum SudokuDifficulty
{
    Easy,
    Medium,
    Hard
}

public static class SudokuHelper
{
    private const string PUZZLES_FOLDER_NAME = "Puzzles";

    public static List<SudokuGrid> GetSudokus(SudokuDifficulty difficulty)
    {
        string fileName = difficulty switch
        {
            SudokuDifficulty.Easy => "Sudoku_Easy51.txt",
            SudokuDifficulty.Medium => "Sudoku_hardest.txt",
            _ => "Sudoku_top95.txt"
        };

        var currentDirectory = new DirectoryInfo(Environment.CurrentDirectory);
        DirectoryInfo puzzlesDirectory = null;

        while (puzzlesDirectory == null)
        {
            puzzlesDirectory = currentDirectory.GetDirectories()
                                               .FirstOrDefault(d => d.Name == PUZZLES_FOLDER_NAME);
            currentDirectory = currentDirectory.Parent;
            if (currentDirectory == null)
                throw new ApplicationException("Couldn't find puzzles directory");
        }

        string filePath = Path.Combine(puzzlesDirectory.ToString(), fileName);
        return SudokuGrid.ReadSudokuFile(filePath);
    }

    public static SudokuGrid SolveSudoku(SudokuGrid sudoku, ISudokuSolver solver)
    {
        display($"Résolution par le solver {solver.GetType().Name} du Sudoku:\n {sudoku}");
        var stopwatch = System.Diagnostics.Stopwatch.StartNew();
        var solvedSudoku = solver.Solve(sudoku);
        stopwatch.Stop();
        display($"Sudoku renvoyé:\n{solvedSudoku}\nNombre d'erreurs réstantes: {solvedSudoku.NbErrors(sudoku)}\nTemps de résolution: {stopwatch.Elapsed.TotalMilliseconds} ms");
        return solvedSudoku;
    }
}


## Affichage des Puzzles de chaque Difficulté

Nous allons charger et afficher un puzzle de chaque niveau de difficulté : Facile, Moyen et Difficile.


In [ ]:
// Chargement et affichage d'un puzzle facile
var easySudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Easy).FirstOrDefault();
display($"Puzzle Facile:\n{easySudoku}");

// Chargement et affichage d'un puzzle moyen
var mediumSudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Medium).FirstOrDefault();
display($"Puzzle Moyen:\n{mediumSudoku}");

// Chargement et affichage d'un puzzle difficile
var hardSudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Hard).FirstOrDefault();
display($"Puzzle Difficile:\n{hardSudoku}");